In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import ee, eemont
from forestry_carbon_arr.core import ForestryCarbonARR
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr

import gcsfs
import os

fs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')


forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
forestry.initialize_gee()

✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


In [3]:
# aoi
from forestry_carbon_arr.core.utils import DataUtils
import geopandas as gpd
import geemap

data_utils = DataUtils(forestry.config, use_gee=True)
aoi_gpd, aoi_ee = data_utils.load_geodataframe_gee(forestry.config["AOI_path"])

aoi_gpd_utm = aoi_gpd.to_crs(epsg=32749)

print(f"✅ AOI loaded: {len(aoi_gpd_utm)} features")
print(f"   Area: {aoi_gpd_utm.geometry.area.sum()/10000:.2f} hectares")

/opt/venv/lib/python3.12/site-packages/geemap/conversion.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✅ AOI loaded: 1 features
   Area: 144217.67 hectares


In [ ]:
pl_dir_apr27 = forestry.config.get('pl_dir_apr27')
pl_dir_jun8 = forestry.config.get('pl_dir_jun8')
pl_dir_jul1 = forestry.config.get('pl_dir_jul1')
pl_dir_jul30 = forestry.config.get('pl_dir_jul30')
pl_dir_sep21_2024 = forestry.config.get('pl_dir_sep21_2024')

In [ ]:
# pl_dir_jul1

In [ ]:
sep_21_2024_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_sep21_2024, "composite_file_format.tif"))
apr_27_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_apr27, "composite_file_format.tif"))
jun_8_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jun8, "composite_file_format.tif"))
jul_1_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul1, "composite_file_format.tif"))
jul_30_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul30,"composite_file_format.tif"))


In [ ]:
apr_27_img.bandNames().getInfo()

In [ ]:
# from wmts_manager import WMTSManager

# wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry(), clear_cache_first=True)
# wmts.addLayer(jul_30_img, {'bands': ['B5', 'B7', 'B3'],
#    'min': 0,
#    'max': 6000,
#    'gamma': 1.5}, 'jul_30_img'
#              )

# wmts.publish()

In [ ]:
## cloud masking start
udm_sep21_2024 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_sep21_2024, "composite_udm2_file_format.tif"))
udm_apr27 =  ee.Image.loadGeoTIFF(os.path.join(pl_dir_apr27, "composite_udm2_file_format.tif"))
udm_jun8 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jun8, "composite_udm2_file_format.tif"))
udm_jul1 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul1, "composite_udm2_file_format.tif"))
udm_jul30 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul30, "composite_udm2_file_format.tif"))


In [ ]:
udm_apr27.bandNames().getInfo()

In [ ]:
udm_jun8.bandNames().getInfo()

In [ ]:
### two band to check in UDM
# condition no cloud

confidence_threshold = 40  # >70 is clear
#using other


def cloud_masking(img, udm_img, confidence_threshold):
    """
    Apply cloud masking using UDM with explicit reprojection.
    """
    # Create masks
    clear_val = 1
    no_snow_val = 0
    no_shadow_val = 0
    no_haze_light_val = 0
    no_haze_heavy_val = 0
    no_cloud_val = 0

    clear_mask = udm_img.select('clear').eq(clear_val)
    no_cloud = udm_img.select('snow').eq(no_snow_val)
    no_shadow = udm_img.select('shadow').eq(no_shadow_val)
    no_haze_light = udm_img.select('haze_light').eq(no_haze_light_val)
    no_haze_heavy = udm_img.select('haze_heavy').eq(no_haze_heavy_val)
    no_cloud = udm_img.select('cloud').eq(no_cloud_val)
    confidence_mask = udm_img.select('confidence').gt(confidence_threshold)
    
    # combined_mask = clear_mask.And(confidence_mask)
    combined_mask = clear_mask.And(confidence_mask.And(no_shadow.And(no_haze_light.And(no_haze_heavy.And(no_cloud.And(no_cloud))))))
    
    # Reproject UDM mask to match image's first band projection
    # This is critical for proper alignment
    img_first_band = img.select(0)
    combined_mask = combined_mask.reproject(
        crs=img_first_band.projection().crs(),
        scale=img_first_band.projection().nominalScale()
    )
    
    # Apply mask
    cloudless_img = img.updateMask(combined_mask)
    
    return cloudless_img

sep_21_2024_img_cloudless = cloud_masking(sep_21_2024_img, udm_sep21_2024, confidence_threshold)
apr_27_img_cloudless = cloud_masking(apr_27_img, udm_apr27, confidence_threshold)
jun_8_img_cloudless = cloud_masking(jun_8_img, udm_jun8, confidence_threshold)
jul_1_img_cloudless = cloud_masking(jul_1_img, udm_jul1, confidence_threshold)
jul_30_img_cloudless = cloud_masking(jul_30_img, udm_jul30, confidence_threshold)

In [ ]:
jul_1_img_cloudless.propertyNames().getInfo()

In [ ]:
from wmts_manager import WMTSManager

wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry(), clear_cache_first=True)

wmts.addLayer(sep_21_2024_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'sep_21_2024_img_cloudless'
             )

wmts.addLayer(apr_27_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'apr_27_img_cloudless'
             )

wmts.addLayer(jun_8_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jun_8_img_cloudless'
             )

wmts.addLayer(jul_1_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jul_1_img_cloudless'
             )

wmts.addLayer(jul_30_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jul_30_img_cloudless'
             )

wmts.publish()

In [ ]:
### based on the images converted to the gee object, we will merge them into gee image collection 
sep_21_2024_img_cloudless = sep_21_2024_img_cloudless.set('system:time_start', ee.Date.fromYMD(2024, 9, 21).millis())
apr_27_img_cloudless = apr_27_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 4, 27).millis())
jun_8_img_cloudless = jun_8_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 6, 8).millis())
jul_1_img_cloudless= jul_1_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 7, 1).millis())
jul_30_img_cloudless = jul_30_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 7, 30).millis())

list_ee_image = [sep_21_2024_img_cloudless, apr_27_img_cloudless, jun_8_img_cloudless, jul_1_img_cloudless, jul_30_img_cloudless]
planet_scope_col= ee.ImageCollection(list_ee_image)

In [ ]:
planet_scope_col.first().get('system:band_names').getInfo()

In [ ]:
planet_scope_col.first().get('system:time_start')

In [ ]:
from forestry_carbon_arr.utils import fill_temporal_gaps_linear

# Use on your SG-smoothed collection
bands_to_fill = ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']  # or the bands you smoothed
planet_scope_col_interpolated = fill_temporal_gaps_linear(planet_scope_col, bands_to_fill)

print('Original monthly (with NaNs):', planet_scope_col.size().getInfo())
print('Filled monthly collection:', planet_scope_col_interpolated.size().getInfo())

In [ ]:
recent_img = planet_scope_col_interpolated.filter(
    ee.Filter.eq('system:time_start', ee.Date.fromYMD(2025, 7, 30).millis())
).first()

wmts.addLayer(recent_img, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'recent_img_FIX'
             )

wmts.publish()

In [ ]:
import ee

# Example: Export single image to GCS
def export_image_to_gcs(image, gcs_bucket, gcs_path, scale=10, crs='EPSG:4326', region=None, max_pixels=1e13):
    """
    Export Earth Engine Image to Google Cloud Storage
    
    Parameters:
    -----------
    image : ee.Image
        Earth Engine Image to export
    gcs_bucket : str
        GCS bucket name (e.g., 'my-bucket' or 'gs://my-bucket')
    gcs_path : str
        Path within bucket (e.g., 'exports/fcd_2020.tif')
    scale : float
        Pixel scale in meters (default: 30)
    crs : str
        Coordinate reference system (default: 'EPSG:4326')
    region : ee.Geometry, optional
        Region to export (default: None, uses image bounds)
    max_pixels : int
        Maximum pixels to export (default: 1e9)
    
    Returns:
    --------
    ee.batch.Task : Export task
    """
    # Clean bucket name (remove gs:// if present)
    if gcs_bucket.startswith('gs://'):
        gcs_bucket = gcs_bucket.replace('gs://', '').split('/')[0]
    
    # Full GCS path
    gcs_uri = f"gs://{gcs_bucket}/{gcs_path}"
    
    # Export parameters
    export_params = {
        'image': image,
        'description': gcs_path.split('/')[-1].replace('.tif', ''),  # Task name
        'bucket': gcs_bucket,
        'fileNamePrefix': gcs_path.replace('.tif', ''),  # Path without extension
        'scale': scale,
        'crs': crs,
        'maxPixels': max_pixels,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {
            'cloudOptimized': True  # COG format
        }
    }
    
    # Add region if provided
    if region is not None:
        export_params['region'] = region
    
    # Create export task
    task = ee.batch.Export.image.toCloudStorage(**export_params)
    
    # Start the task
    task.start()
    
    print(f"✅ Export task started: {gcs_uri}")
    print(f"   Task ID: {task.id}")
    
    return task

In [ ]:
task = export_image_to_gcs(
        image=recent_img,
        gcs_bucket='remote_sensing_saas',
        gcs_path=f'01-korindo/planet_merged/planet_recent_2025_07_interpolated_2024_9_2025_7.tif',
        scale=10,
        crs='EPSG:32749',  # UTM zone for your AOI
        region=aoi_ee.geometry()  # Optional: clip to AOI
    )

In [4]:
## after exported -->
input_image = ee.Image.loadGeoTIFF(os.path.join(f'gs://remote_sensing_saas',f'01-korindo/planet_merged/planet_recent_2025_07_interpolated_2024_9_2025_7.tif'))
input_image.propertyNames().getInfo()    

['system:bands', 'system:band_names']

In [5]:
input_image.bandNames().getInfo()

['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [6]:
input_image_fix = input_image.select(['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']). \
                                rename(['coastal_blue','blue','green1','green','yellow','red','redEdge','nir'])
input_image_fix.bandNames().getInfo()

['coastal_blue', 'blue', 'green1', 'green', 'yellow', 'red', 'redEdge', 'nir']

In [7]:
# forestry.config

In [8]:
# Set I_satellite to 'Custom' in your config
forestry.config['I_satellite'] = 'Custom'

# Run eligibility with your custom image
el = forestry.run_eligibility(
    config=forestry.config,
    use_gee=True,
    custom_image=input_image_fix  # Your pre-processed image
)

processing AVI
processing BSI
processing SI
Normalizing to 100 AVI
Normalizing to 100 AVI
Normalizing to 100 BSI
Normalizing to 100 SI
Combining AVI AND BSI
no thermal band, choosing Planet images
Processing means center of AVI_BSI please wait
Now we proceed to the PCA of Vegetation density
Success get the PCA normalized of VD => SVI
Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2
finish processing PCA, the result: FCD1_1 and FCD2_1 please continue
snic list bands: ['red_mean', 'green_mean', 'blue_mean', 'nir_mean', 'ndwi_mean', 'msavi2_mean', 'MTVI2_mean', 'NDVI_mean', 'VARI_mean', 'FCD1_1_mean', 'FCD2_1_mean', 'area', 'clusters_min', 'width', 'height']


Workflow failed: 'open_land'
Traceback (most recent call last):
  File "/usr/src/app/forestry_carbon_arr/core/main.py", line 942, in run_eligibility
    class_assigning_fcd = AssignClassZone(
                          ^^^^^^^^^^^^^^^^
  File "/usr/src/app/gee_lib/osi/classifying/assign_zone.py", line 22, in __init__
    self.open_land = config['open_land']
                     ~~~~~~^^^^^^^^^^^^^
KeyError: 'open_land'


ForestryCarbonError: Forestry ARR Eligibility analysis failed: 'open_land'